In [ ]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 533.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 11.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.2
    Uninstalling scipy-1.15.2:
      Successfully uninstalled scipy-1.15.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
!pip install swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16505 sha256=806abccae666ae7742d88df5bd668c3e27757d2c773d4474e1391a296bf40365
  Stored in directory: /root/.cache/pip/wheels/ef/7f/bd/9bed48f078f3ee1fa75e0b29b6e0335ce1cb03a38d3443b3a3
Successfully built swifter


In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

import swifter

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#Preprocessing
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r"[^a-z\s]", '', text)
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in stop_words]
    return tokens

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("saurabhshahane/fake-news-classification")

print("Path to dataset files:", path)

100%|██████████| 92.1M/92.1M [00:00<00:00, 182MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/saurabhshahane/fake-news-classification/versions/77


In [ ]:
dataset_path = "/root/.cache/kagglehub/datasets/saurabhshahane/fake-news-classification/versions/77/WELFake_Dataset.csv"
# dataset_path = "/kaggle/input/fake-news-classification/WELFake_Dataset.csv"
df = pd.read_csv(dataset_path)

In [ ]:
df['text'] = df['title'].fillna('') + ' ' + df['text'].fillna('') # Menggabungkan tittle dan text

In [ ]:
df['tokens'] = df['text'].swifter.apply(preprocess_text)# Preprocessing dilakukan

Pandas Apply:   0%|          | 0/72134 [00:00<?, ?it/s]

In [ ]:
X_tokens = df['tokens']
y = df['label'].values

In [ ]:
# Split data
X_train_tokens, X_temp_tokens, y_train, y_temp = train_test_split(X_tokens, y, test_size=0.2, shuffle=False)
X_val_tokens, X_test_tokens, y_val, y_test = train_test_split(X_temp_tokens, y_temp, test_size=0.5, shuffle=False)

In [ ]:
#Word2Vec dan Embedding
# w2v_model = Word2Vec(sentences=df['tokens'], vector_size=100, window=5, min_count=2, workers=4) # Melatih model hasil Preprocessing ke Word2Vec
w2v_model = Word2Vec(sentences=X_train_tokens, vector_size=100, window=5, min_count=2, workers=4)

# Merubah teks menjadi urutan angka
tokenizer = Tokenizer()
# tokenizer.fit_on_texts(df['tokens'])
# sequences = tokenizer.texts_to_sequences(df['tokens'])
tokenizer.fit_on_texts(X_train_tokens)
X_train_seq = tokenizer.texts_to_sequences(X_train_tokens)
X_val_seq = tokenizer.texts_to_sequences(X_val_tokens)
X_test_seq = tokenizer.texts_to_sequences(X_test_tokens)

# Menghitung banyak vocab serta panjang maks
# word_index = tokenizer.word_index
# vocab_size = len(word_index) + 1
# max_len = 100

# Generalisasi ukuran panjang pad
# X = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')
# y = df['label'].values

max_len = 100
X_train = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_val = pad_sequences(X_val_seq, maxlen=max_len, padding='post', truncating='post')
X_test = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')

word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

# Membuat embedding matrix dari Word2Vec
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

In [ ]:
# # Split data
# X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, shuffle=False)
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)

# Biderectional LSTM
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=100,
                    weights=[embedding_matrix], input_length=max_len, trainable=False))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Mengumpulkan model dengan binary crossentropy dan optimizer Adam
model.compile(loss='binary_crossentropy', optimizer=Adam(1e-5), metrics=['accuracy'])
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=64) # Melatih model

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
372/902 ━━━━━━━━━━━━━━━━━━━━ 5:57 675ms/step - accuracy: 0.5470 - loss: 0.6847

KeyboardInterrupt: 

In [ ]:
# # Prediksi data train dan test
y_pred_train = (model.predict(X_train) > 0.5).astype(int)
y_pred_test = (model.predict(X_test) > 0.5).astype(int)

# # Metrik Evaluasi
def get_metrics(y_true, y_pred):
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'F1-Score': f1_score(y_true, y_pred)
    }

# Hitung data latih dan uji
train_metrics = get_metrics(y_train, y_pred_train)
test_metrics = get_metrics(y_test, y_pred_test)

# Hasil Klasifikasi
print("Hasil:")
print(classification_report(y_test, y_pred_test, target_names=["Real", "Fake"]))

In [ ]:
# accuracy = accuracy_score(y_test, y_pred_test)
# precision = precision_score(y_test, y_pred_test)
# recall = recall_score(y_test, y_pred_test)
# f1 = f1_score(y_test, y_pred_test)
# conf_matrix = confusion_matrix(y_test, y_pred_test)

# # y_train_pred = model.predict(X_train_vectorized)
# train_accuracy = accuracy_score(y_train, y_pred_train)
# print(f"Train Accuracy: {train_accuracy:.4f}")

# print(f"Accuracy: {accuracy:.4f}")
# print(f"Precision: {precision:.4f}")
# print(f"Recall: {recall:.4f}")
# print(f"F1 Score: {f1:.4f}")

# class_labels = np.unique(y_test)

# plt.figure(figsize=(8, 6))
# sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
# plt.title('Confusion Matrix Heatmap')
# plt.xlabel('Predicted Label')
# plt.ylabel('True Label')
# plt.show()

In [ ]:
# Grafik Hasil
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Grafik perbandingan hasil Train dan Test
labels = list(train_metrics.keys())
train_vals = list(train_metrics.values())
test_vals = list(test_metrics.values())

x = np.arange(len(labels))
width = 0.35

plt.figure(figsize=(8, 5))
plt.bar(x - width/2, train_vals, width, label='Train')
plt.bar(x + width/2, test_vals, width, label='Test')
plt.xticks(x, labels)
plt.ylabel('Score')
plt.title('Train vs Test Metrics')
plt.ylim(0, 1)
plt.legend()
plt.show()